In [15]:
import os
#from pytorch_tabnet.tab_model import TabNetClassifier
#from numba import jit, cuda
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
#from imblearn.over_sampling import SMOTE
#from imblearn.over_sampling import SMOTENC
from sklearn.utils import shuffle
from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn import tree
#from xgboost import plot_tree
import pandas as pd
import numpy as np
import os
#import wget
from pathlib import Path
import shutil
import gzip
import joblib
import pydot

random_state=42
np.random.seed(random_state)

from matplotlib import pyplot as plt
from numpy.random import randn

import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


from tabgan.sampler import OriginalGenerator, GANGenerator

In [24]:
df_test = pd.read_csv('../data/EdgeIIot_test.csv', low_memory=False)
df_train = pd.read_csv('../data/DNN-EdgeIIoT-dataset_SMALL.csv', low_memory=False)

drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
         "tcp.dstport", "udp.port", "mqtt.msg"]

df_train.drop(drop_columns, axis=1, inplace=True)

df_train.drop(["Attack_label"], axis=1, inplace=True)
df_test.drop(["Attack_label"], axis=1, inplace=True)

features = [ col for col in df_train.columns if col not in ["Attack_label"]+["Attack_type"]] 

featuresFromStart = [ col for col in df_train.columns if col not in ["Attack_type"]]
categorical_columns = []
for col in df_train.columns[df_train.dtypes == object]:
    if col != "Attack_type":
        categorical_columns.append(col)
        
catIndexs = []
for cc in categorical_columns:
    catIndexs.append(featuresFromStart.index(cc))
    

df = pd.concat([df_train,df_test],keys=[0,1])

colunas_one_hot = {}
for coluna in categorical_columns:
    codes, uniques = pd.factorize(df[coluna].unique())
    colunas_one_hot[coluna] = {"uniques": uniques, "codes":codes}
    df[coluna] = df[coluna].replace(colunas_one_hot[coluna]["uniques"], colunas_one_hot[coluna]["codes"])

df = pd.get_dummies(data=df, columns=categorical_columns)

df_train,df_test = df.xs(0),df.xs(1)

features = [ col for col in df_train.columns if col not in ["Attack_label"]+["Attack_type"]] 

categorical_num = [item for item in features if item not in featuresFromStart]
print(categorical_num)

catIndexs_num = []
for cc in categorical_num:
    catIndexs_num.append(features.index(cc))

"""

features = [ col for col in df_train.columns if col not in ["Attack_label"]+["Attack_type"]] 

le = LabelEncoder()
le.fit(df_train["Attack_type"].values)

X_train = df_train[features].values
y_train = df_train["Attack_type"].values
y_train = le.transform(y_train)

X_test = df_test[features].values
y_test = df_test["Attack_type"].values
y_test = le.transform(y_test)

standScaler = StandardScaler()
model_norm = standScaler.fit(X_train)

X_train = model_norm.transform(X_train)
X_test = model_norm.transform(X_test)

"""
le = LabelEncoder()
le.fit(df_train["Attack_type"].values)

y_train = df_train["Attack_type"].values
df_y_train = pd.DataFrame(le.transform(y_train), columns=["Attack_type"])
df_train.drop(["Attack_type"], axis=1, inplace=True)

X_test = df_test[features].values
y_test = df_test["Attack_type"].values

df_test_x = pd.DataFrame(X_test, columns=features)
df_y_test = pd.DataFrame(le.transform(y_test), columns=["Attack_type"])



['http.request.method_0', 'http.request.method_1', 'http.request.method_2', 'http.request.method_3', 'http.request.method_4', 'http.request.method_5', 'http.request.method_6', 'http.request.method_7', 'http.referer_0', 'http.referer_1', 'http.referer_2', 'http.referer_3', 'http.request.version_0', 'http.request.version_1', 'http.request.version_2', 'http.request.version_3', 'http.request.version_4', 'http.request.version_5', 'http.request.version_6', 'http.request.version_7', 'dns.qry.name.len_0', 'dns.qry.name.len_1', 'dns.qry.name.len_2', 'dns.qry.name.len_3', 'dns.qry.name.len_4', 'dns.qry.name.len_5', 'dns.qry.name.len_6', 'dns.qry.name.len_7', 'dns.qry.name.len_8', 'dns.qry.name.len_9', 'mqtt.conack.flags_0', 'mqtt.conack.flags_1', 'mqtt.conack.flags_2', 'mqtt.conack.flags_3', 'mqtt.conack.flags_4', 'mqtt.conack.flags_5', 'mqtt.conack.flags_6', 'mqtt.protoname_0', 'mqtt.protoname_1', 'mqtt.protoname_2', 'mqtt.topic_0', 'mqtt.topic_1', 'mqtt.topic_2']


/Users/melicias/.local/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
# generate data
#new_train1, new_target1 = OriginalGenerator().generate_data_pipe(
#    pd.DataFrame(X_train, columns=features), pd.DataFrame(y_train, columns=["Attack_type"]), pd.DataFrame(X_test, columns=features), )
print(categorical_num)
new_train2, new_target2 = GANGenerator(cat_cols=categorical_num).generate_data_pipe(df_train, df_y_train, df_test_x, )

# example with all params defined
"""
categorical_columns
new_train3, new_target3 = GANGenerator(gen_x_times=0.5, cat_cols=categorical_columns,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "AUC", "max_depth": 2, "max_bin": 50, 
               "learning_rate": 0.05, "random_state": 42, "n_estimators": 100,
           }, pregeneration_frac=2, only_generated_data=False,
           gan_params = {"batch_size": 64, "patience": 25, "epochs" : 200,}).generate_data_pipe(df_train, df_y_train, df_test_x,
                                                            deep_copy=True, only_adversarial=False, use_adversarial=True)
"""

['http.request.method_0', 'http.request.method_1', 'http.request.method_2', 'http.request.method_3', 'http.request.method_4', 'http.request.method_5', 'http.request.method_6', 'http.request.method_7', 'http.referer_0', 'http.referer_1', 'http.referer_2', 'http.referer_3', 'http.request.version_0', 'http.request.version_1', 'http.request.version_2', 'http.request.version_3', 'http.request.version_4', 'http.request.version_5', 'http.request.version_6', 'http.request.version_7', 'dns.qry.name.len_0', 'dns.qry.name.len_1', 'dns.qry.name.len_2', 'dns.qry.name.len_3', 'dns.qry.name.len_4', 'dns.qry.name.len_5', 'dns.qry.name.len_6', 'dns.qry.name.len_7', 'dns.qry.name.len_8', 'dns.qry.name.len_9', 'mqtt.conack.flags_0', 'mqtt.conack.flags_1', 'mqtt.conack.flags_2', 'mqtt.conack.flags_3', 'mqtt.conack.flags_4', 'mqtt.conack.flags_5', 'mqtt.conack.flags_6', 'mqtt.protoname_0', 'mqtt.protoname_1', 'mqtt.protoname_2', 'mqtt.topic_0', 'mqtt.topic_1', 'mqtt.topic_2']


Fitting CTGAN transformers for each column:   0%|          | 0/83 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

: 

: 

In [ ]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    
    return X, y

def generate_real_samples(n):
    X = df_train.sample(n)
    y = X["Attack_type"]
    X.drop(["Attack_type"], axis=1, inplace=True)
    return X, y

In [ ]:
def define_generator(latent_dim, n_outputs):
    
    model = Sequential()
    model.add(Dense(32, input_dim=latent_dim, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    #model.add(Dense(n_outputs, activation='softmax')) 
    model.add(Dense(n_outputs, activation='linear'))
    return model

def define_discriminator(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def plot_history(d_hist, g_hist):
    # plot loss
    plt.subplot(1, 1, 1)
    plt.plot(d_hist, label='d')
    plt.plot(g_hist, label='gen')
    plt.show()
    plt.close()

In [ ]:
generator1 = define_generator(X_train.shape[1], len(le.classes_))
generator1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_150 (Dense)            (None, 32)                3072      
_________________________________________________________________
batch_normalization_v1_50 (B (None, 32)                128       
_________________________________________________________________
dropout_50 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_151 (Dense)            (None, 64)                2112      
_________________________________________________________________
batch_normalization_v1_51 (B (None, 64)                256       
_________________________________________________________________
dropout_51 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_152 (Dense)            (None, 15)                975       
Total para

In [ ]:
discriminator1 = define_discriminator(len(le.classes_),X_train.shape[1])
discriminator1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_153 (Dense)            (None, 64)                1024      
_________________________________________________________________
dense_154 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_155 (Dense)            (None, 95)                3135      
Total params: 6,239
Trainable params: 6,239
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=200):
    # determine half the size of one batch, for updating the  discriminator
    half_batch = int(n_batch / 2)
    d_history = []
    g_history = []
    # manually enumerate epochs
    for epoch in range(n_epochs):
    
        # prepare real samples
        x_real, y_real = generate_real_samples(half_batch)
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        print(np.shape(x_real),np.shape(y_real))
        print(d_model.summary())
        # update discriminator
        d_loss_real, d_real_acc = d_model.train_on_batch(x_real, y_real)
        d_loss_fake, d_fake_acc = d_model.train_on_batch(x_fake, y_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        # update the generator via the discriminator's error
        g_loss_fake = gan_model.train_on_batch(x_gan, y_gan)
        print('>%d, d1=%.3f, d2=%.3f d=%.3f g=%.3f' % (epoch+1, d_loss_real, d_loss_fake, d_loss,  g_loss_fake))
        d_history.append(d_loss)
        g_history.append(g_loss_fake)
        
    plot_history(d_history, g_history)    
    g_model.save('trained_generated_model.h5')

In [ ]:
print(X_train.shape[1])
print(len(le.classes_)) # 15

# size of the latent space
latent_dim = X_train.shape[1] #95
# create the discriminator
discriminator = define_discriminator(latent_dim, len(le.classes_))
discriminator.summary()
# create the generator
generator = define_generator(len(le.classes_), latent_dim)
generator.summary()
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, len(le.classes_))

95
15
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_156 (Dense)            (None, 64)                6144      
_________________________________________________________________
dense_157 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_158 (Dense)            (None, 15)                495       
Total params: 8,719
Trainable params: 8,719
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_159 (Dense)            (None, 32)                512       
_________________________________________________________________
batch_normalization_v1_52 (B (None, 32)                128       
_________________________________________________________________
dr

ValueError: A target array with shape (64, 1) was passed for an output of shape (None, 15) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.